In [1]:
import pandas as pd

df = pd.read_json("../../datas/nikto/detected.json", lines=True)
# df = pd.read_json("../detected.json", lines=True)

df.head(5)

,method,path,headers,uuid,peer,status,cookies,response_msg,timestamp,post_data
0,GET,/,"{'host': '127.0.0.1', 'user-agent': 'curl/7.81...",eee6754c-7a79-4143-b40a-fb4f6358f230,"{'ip': '172.18.0.1', 'port': 36070}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:02:40.155204,NaN
1,HEAD,/,"{'host': 'localhost', 'connection': 'Keep-Aliv...",eee6754c-7a79-4143-b40a-fb4f6358f230,"{'ip': '172.18.0.1', 'port': 58816}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:03:06.043339,NaN
2,GET,/,{'user-agent': 'Mozilla/5.00 (Nikto/2.1.5) (Ev...,eee6754c-7a79-4143-b40a-fb4f6358f230,"{'ip': '172.18.0.1', 'port': 58816}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:03:06.107540,NaN
3,GET,/,{'user-agent': 'Mozilla/5.00 (Nikto/2.1.5) (Ev...,eee6754c-7a79-4143-b40a-fb4f6358f230,"{'ip': '172.18.0.1', 'port': 58816}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:03:06.116424,NaN
4,GET,/bTsJgT9V,{'user-agent': 'Mozilla/5.00 (Nikto/2.1.5) (Ev...,eee6754c-7a79-4143-b40a-fb4f6358f230,"{'ip': '172.18.0.1', 'port': 58816}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:03:06.126323,NaN


In [2]:
attacks_matrix = []
tmp = []
sess_uuids_attacks = {
  # "{sess_uuid}": []
}
sess_uuids = []

for _, row in df.iterrows():
  sess_uuid = row.get("cookies", {}).get("sess_uuid") or row.get("response_msg", {}).get("response", {}).get("message", {}).get("sess_uuid")
  detection = row.get("response_msg", {}).get("response", {}).get("message", {}).get("detection", {})
  attack_type = detection.get("name", "")

  if attack_type == "index":
    continue

  if sess_uuid in sess_uuids:
    sess_uuids_attacks[sess_uuid].append(attack_type)
  else:
    sess_uuids_attacks[sess_uuid] = [attack_type]
    sess_uuids.append(sess_uuid)

attacks_matrix = list(sess_uuids_attacks.values())
# for paths in sess_uuids_attacks.values():
#   attacks_matrix.append(paths)

sess_uuids_attacks
# attacks_matrix
# print(len(attacks_matrix), len(sess_uuids_paths.keys()))

{'4088b241-7999-4b75-8a02-759c7cd6d680': ['cmd_exec', 'cmd_exec', 'cmd_exec'],
 'c7427370-2786-4139-8fe4-11c3046619b6': ['cmd_exec'],
 'ca1f7d79-d7f7-4dfe-a5ec-12ba0b697f68': ['cmd_exec'],
 'd1bf577e-6adc-4396-baed-72cb2876242b': ['cmd_exec'],
 '1d193001-7608-4c7a-9c97-ddf77496dc02': ['cmd_exec'],
 '2347f57a-1449-42c8-abbf-799be5328200': ['cmd_exec'],
 '0cd704f5-eab2-48db-a197-a061997ad871': ['cmd_exec'],
 '997f500e-b3a1-4de5-9ab8-100d110f81d3': ['xss'],
 'f8fa4257-4d7c-4c04-b749-fd14e5897760': ['xss'],
 'ba0af8f4-5023-4ad2-b33c-f779027046a1': ['xss'],
 'c90931e2-0c48-44ca-b469-0658706e35da': ['wp-content', 'wp-content'],
 '30125182-64ff-4f04-b728-d997fdd13024': ['unknown'],
 '6ddca7c6-8789-47e0-a02b-eece5e36534f': ['unknown'],
 'e6b8cb6d-c1d5-4a43-900f-25f5b6c2895f': ['lfi'],
 '3c9e4588-44e5-4d49-8ce7-3381f061d7c8': ['xss'],
 'f7e0674b-3193-45e1-a20a-cc50a5a0a3ab': ['xss'],
 '9db2d793-71fe-4292-b84b-e08398722e1f': ['unknown'],
 '52304b86-465f-40a6-9b23-af513719af9c': ['xss'],
 '08a15c

In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

te = TransactionEncoder()
te_ary = te.fit(attacks_matrix).transform(attacks_matrix)
df = pd.DataFrame(te_ary, columns=te.columns_)

df.head(10)

,cmd_exec,lfi,sqli,unknown,wp-content,xss
0,True,False,False,False,False,False
1,True,False,False,False,False,False
2,True,False,False,False,False,False
3,True,False,False,False,False,False
4,True,False,False,False,False,False
5,True,False,False,False,False,False
6,True,False,False,False,False,False
7,False,False,False,False,False,True
8,False,False,False,False,False,True
9,False,False,False,False,False,True


In [ ]:
# frequent_itemsets = fpgrowth(df, min_support=0.3, use_colnames=True)
frequent_itemsets = fpgrowth(df, min_support=0.00001, use_colnames=True)
frequent_itemsets["length"] = frequent_itemsets["itemsets"].apply(lambda x: len(x))
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

frequent_itemsets

,support,itemsets,length
0,0.056548,(cmd_exec),1
1,0.791667,(xss),1
2,0.008929,(wp-content),1
3,0.017857,(unknown),1
4,0.113095,(lfi),1
5,0.011905,(sqli),1


In [8]:
large_frequent_itemsets = frequent_itemsets[frequent_itemsets["length"] >= 3]
large_frequent_itemsets

,support,itemsets,length


In [13]:
import psycopg2

conn = psycopg2.connect(database="web_honeypot_nikto", user = "postgres", password = "admin", host = "127.0.0.1", port = "5432")

print("Opened database successfully")

Opened database successfully


In [14]:
# create table
cur = conn.cursor()
cur.execute('''CREATE TABLE assoc_rules_attack_types (
            ID INT PRIMARY KEY     NOT NULL,
            SUPPORT           REAL    NOT NULL,
            ATTACK_TYPE            VARCHAR(255)     NOT NULL);''')

print("Table created successfully")

conn.commit()

Table created successfully


In [15]:
# insert data

cur = conn.cursor()

insert_query = """
    INSERT INTO assoc_rules_attack_types (ID, SUPPORT, ATTACK_TYPE)
    VALUES (%s, %s, %s)
"""

for idx, row in frequent_itemsets.iterrows():
    cur.execute(
        insert_query,
        (int(idx), float(row["support"]), str(list(row["itemsets"])))
    )

conn.commit()
print("Records created successfully")
conn.close()

Records created successfully


In [9]:
# create large data table
cur = conn.cursor()
cur.execute('''CREATE TABLE assoc_rules_attack_types_large (
            ID INT PRIMARY KEY     NOT NULL,
            SUPPORT           REAL    NOT NULL,
            ATTACK_TYPE            VARCHAR(255)     NOT NULL);''')

print("Table created successfully")

conn.commit()

InterfaceError: connection already closed

In [ ]:
# insert large data

cur = conn.cursor()

insert_query = """
    INSERT INTO assoc_rules_attack_types_large (ID, SUPPORT, ATTACK_TYPE)
    VALUES (%s, %s, %s)
"""

for idx, row in large_frequent_itemsets.iterrows():
    cur.execute(
        insert_query,
        (int(idx), float(row["support"]), str(list(row["itemsets"])))
    )

conn.commit()
print("Records created successfully")
conn.close()

Records created successfully
